In [57]:
import os
import pandas as pd
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import sys
import os
import pickle

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
Year = 1941
Title = "A Woman's Face"
Plot = "in sweden ,  a female blackmailer with a disfiguring facial scar meets a gentleman who lives beyond his means .  they become accomplices in blackmail ,  and she falls in love with him ,  bitterly resigned to the impossibility of his returning her affection .  her life changes when one of her victims proves to be the wife of a plastic surgeon ,  who catches her in his apartment ,  but believes her to be a jewel thief rather than a blackmailer .  he offers her the chance to look like a normal woman again ,  and she accepts ,  despite the agony of multiple operations .  meanwhile ,  her gentleman accomplice forms an evil scheme to rid himself of the one person who stands in his way to a fortune  -  his four - year - old - nephew ."

In [93]:
clf = joblib.load('model_logit.pkl')
with open("vocabulary.txt", "rb") as fp:
    voc_ = pickle.load(fp)
with open("th.txt", "rb") as fp:
    th = pickle.load(fp)

data=pd.DataFrame({'year':[Year],
    'title':[Title],
    'plot':[Plot]})


In [28]:

data['title']=data['title'].str.lower()
data['title']=data['title'].str.replace(',', '') 
data['title']=data['title'].str.replace('.', '') 
data['title']=data['title'].str.replace('-', '') 
data['title']=data['title'].str.replace('_', '') 
data['plot']=data['plot'].str.lower()
data['plot']=data['plot'].str.replace(',', '') 
data['plot']=data['plot'].str.replace('.', '') 
data['plot']=data['plot'].str.replace('-', '') 
data['plot']=data['plot'].str.replace('_', '')
data['uno'] = data['title']+data['plot']
data.drop(['title','plot'], axis=1, inplace=True)

In [29]:
wordnet_lemmatizer = WordNetLemmatizer()
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word, 's') for word in words]


In [30]:
vect = TfidfVectorizer(analyzer=split_into_lemmas,
                        min_df=2,
                        max_features=25564,
                        sublinear_tf=True,
                        strip_accents='unicode',
                        ngram_range=(1,3),
                        stop_words='english',
                        vocabulary=voc_)

In [31]:
X_dtm = vect.fit_transform(data['uno'], data['year'])

In [32]:
X_dtm

<1x25564 sparse matrix of type '<class 'numpy.float64'>'
	with 82 stored elements in Compressed Sparse Row format>

In [142]:
cols = np.array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
    'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Romance',
    'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western']).reshape((24,1))

genres = np.transpose(clf.predict_proba(X_dtm))
genres2 = np.copy(genres)
for i in range(0,len(th)):
    genres2[i] = int(genres2[i]>th[i])

data = np.append(cols, np.round(genres,2), axis=1)
data = np.append(data, genres2, axis=1)
res = pd.DataFrame(data, columns=['Genre','Prob','Val'])
res = res[genres2 != 0]
res.sort_values('Prob', ascending=False, inplace=True)
res.drop(['Val'], axis=1, inplace=True)
res.reset_index(drop=True, inplace=True)

In [143]:
res

,Genre,Prob
0,Drama,0.9
1,Romance,0.86
2,Crime,0.17
3,Mystery,0.12
4,Fantasy,0.09
5,Film-Noir,0.04
6,Musical,0.04


In [107]:
genres

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [92]:
print('Probability table of movie genres')
print()

Probability table of movie genres
       Genre                    Prob
       Drama      0.9022209708338398
     Romance       0.864253965199702
      Comedy      0.2604862733589537
    Thriller     0.23277014038661006
       Crime     0.16735162998101716
     Mystery     0.12124819844487446
     Fantasy     0.08949329024022601
   Adventure    0.053402262921874846
      Horror     0.04365057285038115
   Film-Noir     0.04184276600780621
     Musical     0.04140074323223314
      Family    0.025031455626119647
   Biography      0.0175944264479467
     History     0.01347988239045166
     Western     0.01065461189148406
      Sci-Fi    0.009699401964983633
       Music    0.009135758769246017
         War    0.007589063981345125
   Animation    0.007237020883891858
       Sport    0.007086444038598754
      Action    0.005803930031070418
       Short    0.004372578324937518
        News   0.0006197163853451663
 Documentary  0.00018753345643774015
